# Exploring building virtual datasets with kerchunk

The idea: build a test intake catalogue of kerchunk virtual datasets from the COSIMA outputs and see how it performs

In [1]:
import dask

import glob

import fsspec

import ujson

from distributed import Client

from kerchunk.hdf import SingleHdf5ToZarr

In [2]:
client = Client(n_workers=8)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36631,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:36137,Total threads: 1
Dashboard: http://127.0.0.1:38633/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:34875,


# First try build a virtual dataset from `access-om2-025` `025deg_jra55_iaf_omip2_cycle1` `ocean_month` data (~2.2TB, 61 netcdf files)

## Write single file jsons in parallel

In [3]:
exp_root = "/g/data/ik11/outputs/access-om2-025/025deg_jra55_iaf_omip2_cycle1"

fs = fsspec.filesystem('file')
flist = fs.glob(f"{exp_root}/output*/ocean/ocean_month.nc")

In [6]:
@dask.delayed
def gen_json(file):
    with fs.open(file) as infile:
        h5chunks = SingleHdf5ToZarr(infile, file)
        outf = f"{'.'.join(file.split('/')[5:])}.json"
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [7]:
%%time

# This would take well over an hour without dask
_ = dask.compute(*[gen_json(file) for file in flist])

CPU times: user 1min 10s, sys: 48 s, total: 1min 58s
Wall time: 13min 55s


## Combine into multi-file json

In [8]:
from kerchunk.combine import MultiZarrToZarr

In [11]:
%%time
json_list = fs.glob("./access-om2-025.025deg_jra55_iaf_omip2_cycle1.*.ocean.ocean_month.nc.json")

mzz = MultiZarrToZarr(
    json_list[:2],
    concat_dims=['time'],
    identical_dims=[
        "xt_ocean", 
        "yt_ocean", 
        "st_ocean", 
        "xu_ocean", 
        "yu_ocean", 
        "sw_ocean", 
        "grid_xt_ocean", 
        "grid_yt_ocean", 
        "grid_xu_ocean", 
        "grid_yu_ocean", 
        "potrho", 
        "neutral", 
        "nv"
     ],
)

d = mzz.translate()

with open('access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json', 'wb') as f:
    f.write(ujson.dumps(d).encode())

CPU times: user 2.91 s, sys: 1.47 s, total: 4.38 s
Wall time: 3.08 s


In [18]:
import xarray as xr

ds = xr.open_dataset(fs, engine="zarr", backend_kwargs={
                    "consolidated": False,
                    "storage_options": {"fo": 'access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json'}
                    })
print(ds)

ValueError: Starting with Zarr 2.11.0, stores must be subclasses of BaseStore, if your store exposes the MutableMapping interface wrap it in Zarr.storage.KVStore. Got <fsspec.implementations.local.LocalFileSystem object at 0x145cf1b596a0>